In [1]:
!pip install napari

You should consider upgrading via the '/Users/esb/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import matplotlib.patches as patches
from descartes.patch import PolygonPatch
from matplotlib.backends.backend_qt5agg import FigureCanvas
from matplotlib.figure import Figure

import networkx as nx
import pygraphviz
import pydot
from networkx.drawing.nx_agraph import graphviz_layout

import random
import pandas as pd
import napari

import random
from shapely.geometry import Point,MultiPoint,MultiPolygon,Polygon,box,LineString
from shapely import affinity
import re

from shapely import geometry
import pickle
import time

#import ipywidgets as widgets
#import ipywidgets.widgets.interaction
#from ipywidgets import *
import warnings
warnings.simplefilter('ignore', np.RankWarning)

from copy import deepcopy
from magicgui import magicgui
import skimage.measure as measure
from dask_image.imread import imread
#import src

from napari_animation import Animation
from napari.utils import nbscreenshot
from napari_animation import AnimationWidget

from PIL import Image

%load_ext autoreload

/Users/esb/opt/anaconda3/lib/python3.7/site-packages/napari/_qt/__init__.py:37: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [3]:
path_to_uj='../uJ/uJ_src_pBGT/python/'
sys.path.insert(0, path_to_uj)

uj_data_path="../../data/"


In [4]:
#%autoreload 2
##from readroi import *
from DataManagers import *
#from TrackingFunctions import *
#from PlottingFunctions import *
#from AnalysisFunctions import *
##from CorrectorFunctionsG import *

Importing readroi
DataStructs...loaded!
DataManagers...loaded!


In [6]:

frame2min=10

figxlim=640
figylim=512
flims=(figxlim,figylim)


channels=["DIC","GFP","DsRed"]
channels_color=["gray","green","red"]
vis=[False,True,True]
vis=[False,False,False]
seg_chan=channels[1]

max_frames=-1 #All

## Experimental parameters: pBGT

In [7]:
rootDir=uj_data_path+"pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/"

expeLabel="pBGT-AMP-Pulse"

frame_experiment_start=20;
frame_experiment_end=44
frame_signal_start=26
frame_signal_end=34
frame_norm_start=20
frame_norm_end=25
frame_analysis_end=44

n_frames=frame_experiment_end-frame_experiment_start+1

t_experiment_start=0
t_signal_start=(frame_signal_start-frame_experiment_start)*frame2min  
t_signal_end=(frame_signal_end-frame_experiment_start)*frame2min
t_experiment_end=(frame_experiment_end-frame_experiment_start)*frame2min

frames=range(int(frame_experiment_start),int(frame_experiment_end+1))
frames_list=[x for x in frames]
times=[(this_frame-frame_experiment_start)*frame2min for this_frame in frames]
times_hour=[(this_frame-frame_experiment_start)*frame2min/60 for this_frame in frames]

mask_chan=channels[1]
data_type="normData"

expeLabel="pBGT-AMP-Pulse"
rootDir=uj_data_path+"pBGT-Pulse/"

full_trap_list="xy01,xy02,xy03,xy11,xy12,xy13,xy14,xy20,xy21,xy22,xy23,xy25,xy26,xy27,xy28,xy29,xy30,xy33,xy34,xy35,xy36,xy37,xy39,xy40,xy41,xy42,xy43,xy44,xy45,xy46,xy47".split(",")

trap_list=["xy02"]

thres_length=38.2299101979529
thres_DsRed=146.11125000000000

current_experiment="Plasmid"

## Experimental data

In [8]:

dirNameDataLineages=rootDir+'data/lineages/'
dirNameDataCellsTrackedCorrected=rootDir+'data_cells_tracked_corrected/'
dirNameDataLineagesCorrected=rootDir+'data/lineages_corrected/'
dirNameDataNapariLabels=rootDir+'data_napari_labels/'

trap_masks_path=[]
trap_tif_path=[]
for pos in trap_list:
    tif_path=rootDir+"data_raw/"+pos+"/"
    trap_tif_path.append(tif_path)
    masks_path=rootDir+"data_masks/"+pos+"/"+mask_chan+"/"
    trap_masks_path.append(masks_path)
    

### Load segmentation

In [9]:


trap_tracked_cells=[]
trap_tracked_cells_corrected=[]
trap_corrected_list=[]
for trap in trap_list:
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+"/"+data_type+"/"
    
    dirNameTRACKEDCELLS_Corrrected=dirNameDataCellsTrackedCorrected+trap+"/"+data_type+"/"
    iscorrected=os.path.exists(dirNameTRACKEDCELLS_Corrrected)
    trap_corrected_list.append(iscorrected)
    if(iscorrected):
        print("Loading corrected cells for",trap)
        tracked_cells_corrected=load_cells(dirNameTRACKEDCELLS_Corrrected,max_frames)
        trap_tracked_cells_corrected.append(tracked_cells_corrected)
    else:
        print("Loading original cells for",trap)
        tracked_cells=load_cells(dirNameTRACKEDCELLS,max_frames)
        trap_tracked_cells.append(tracked_cells)
    
        tracked_cells_corrected=tracked_cells.copy()
        trap_tracked_cells_corrected.append(tracked_cells_corrected)
    

Loading corrected cells for xy02
0: Loading pBGT-AMP-Pulse_xy02_020.pkl
1: Loading pBGT-AMP-Pulse_xy02_021.pkl
2: Loading pBGT-AMP-Pulse_xy02_022.pkl
3: Loading pBGT-AMP-Pulse_xy02_023.pkl
4: Loading pBGT-AMP-Pulse_xy02_024.pkl
5: Loading pBGT-AMP-Pulse_xy02_025.pkl
6: Loading pBGT-AMP-Pulse_xy02_026.pkl
7: Loading pBGT-AMP-Pulse_xy02_027.pkl
8: Loading pBGT-AMP-Pulse_xy02_028.pkl
9: Loading pBGT-AMP-Pulse_xy02_029.pkl
10: Loading pBGT-AMP-Pulse_xy02_030.pkl
11: Loading pBGT-AMP-Pulse_xy02_031.pkl
12: Loading pBGT-AMP-Pulse_xy02_032.pkl
13: Loading pBGT-AMP-Pulse_xy02_033.pkl
14: Loading pBGT-AMP-Pulse_xy02_034.pkl
15: Loading pBGT-AMP-Pulse_xy02_035.pkl
16: Loading pBGT-AMP-Pulse_xy02_036.pkl
17: Loading pBGT-AMP-Pulse_xy02_037.pkl
18: Loading pBGT-AMP-Pulse_xy02_038.pkl
19: Loading pBGT-AMP-Pulse_xy02_039.pkl
20: Loading pBGT-AMP-Pulse_xy02_040.pkl
21: Loading pBGT-AMP-Pulse_xy02_041.pkl
22: Loading pBGT-AMP-Pulse_xy02_042.pkl
23: Loading pBGT-AMP-Pulse_xy02_043.pkl
24: Loading pBGT-

### Load lineages files

In [14]:
trap_df_lineages_all=[]
trap_df_lineages_corrected_all=[]

for i,trap in enumerate(trap_list):
    
    if(trap_corrected_list[i]):
        print("Loading corrected lineages !")
        lineagesDataFile='%s%s_%s_lineages_filtered.csv'%(dirNameDataLineagesCorrected, expeLabel, trap)
        df_lineagesC_all=load_data_lineages(lineagesDataFile)
        trap_df_lineages_corrected_all.append(df_lineagesC_all)
    else:
        lineagesDataFile='%s%s_%s_lineages_all.csv'%(dirNameDataLineages, expeLabel, trap)
        df_lineages_all=load_data_lineages(lineagesDataFile)
        trap_df_lineages_all.append(df_lineages_all)
        print("Loading original lineages !")
        df_lineagesC_all=df_lineages_all.copy()
        trap_df_lineages_corrected_all.append(df_lineagesC_all)
#df_lineages=df_lineages_all

Loading corrected lineages !
Loading 667 lineages from ../../data/pBGT-Pulse/data/lineages_corrected/pBGT-AMP-Pulse_xy02_lineages_filtered.csv


### Add geometric info to dataframe

In [15]:
trap_df_lineages_geoms=[]

for i,trap in enumerate(trap_list):
    if(trap_corrected_list[i]):
        tracked_cells=trap_tracked_cells_corrected[i]
        df_lineages=trap_df_lineages_corrected_all[i]
    else:
        tracked_cells=trap_tracked_cells[i]
        df_lineages=trap_df_lineages_all[i]
    trap_newDF_list=[]
    for row in df_lineages.itertuples(index=False):
        
        this_id=str(row.cellID)
        
        trackid=row.trackID
        f=this_id
        n='0'
        if this_id.count('.')>0:
            f,n=this_id.split('.')
        this_cell=tracked_cells[int(f)-frame_experiment_start][int(n)]
        new_df=pd.DataFrame.from_records([{"cellID":this_id,"trackID":trackid,"cellColor":this_cell['cellColor'],"center":this_cell['center'],"roiPoly":this_cell['roiPoly'],"axis":this_cell['axis']}], index='cellID')
        trap_newDF_list.append(new_df)
    trap_newDF=pd.concat(trap_newDF_list)
    trap_newDF['trackID']=trap_newDF['trackID'].astype(str)
    df_lineages['cellID']=df_lineages['cellID'].astype(str)
    df_lineages_geoms=pd.merge(df_lineages,trap_newDF,on=['cellID','trackID'])
    trap_df_lineages_geoms.append(df_lineages_geoms)
df_lineages_geoms.head(3)

,lineageID,trackID,cellID,motherID,roiID,trapID,frame,length,division,state,tracking_score,GFP,DsRed,cellColor,center,roiPoly,axis
0,1,20.038-44.000,20.038,20.038,roi_f20_n38_x234_y332,xy02,20,75.956580,0,2,87.092595,581.572,98.007,#5ffdc6,POINT (231.7181860965421 331.8348196488547),"POLYGON ((235 334, 236 334, 236 336, 237 336, ...",LINESTRING (249.2708810983069 365.000000000000...
1,1,20.038-44.000,21.032,20.038,roi_f21_n32_x236_y336,xy02,21,82.770104,0,2,104.219936,551.989,97.685,#5ffdc6,POINT (234.0590205989491 336.2403839020085),"POLYGON ((237 338, 238 338, 238 340, 239 340, ...",LINESTRING (255.6909247005913 370.934125597546...
2,1,20.038-44.000,22.028,20.038,roi_f22_n28_x238_y341,xy02,22,91.366660,0,2,-15.568477,574.380,98.031,#5ffdc6,POINT (235.7176928469514 341.0466021609977),"POLYGON ((239 343, 240 343, 240 346, 241 346, ...",LINESTRING (260.3561804634648 378.999999999999...


In [16]:
empty_mask=np.zeros((n_frames,figylim,figxlim))   ### Creates empty matrix array
matrix_shape = empty_mask.shape[1:]
roi_mask=deepcopy(empty_mask)

trap_roi_masks=[]
trap_cellID_to_lineage=[]
#trap_colorsLid=[]
#trap_colorsCid=[]
for ti,trap in enumerate(trap_list):

    print("Loading Napari files...")
    this_file=dirNameDataNapariLabels+"%s_%s_roi_mask.npy"%(trap,seg_chan)
    roi_mask=np.load(this_file)
    this_file=dirNameDataNapariLabels+"%s_%s_colors_dict.npy"%(trap,seg_chan)
    colorsCid=np.load(this_file,allow_pickle=True).item()
              
    this_file=dirNameDataNapariLabels+"%s_%s_colors_lineages_dict.npy"%(trap,seg_chan)
    colorsLid=np.load(this_file,allow_pickle=True).item()
        
    this_file=dirNameDataNapariLabels+"%s_%s_cell_lineages_dict.npy"%(trap,seg_chan)
    cellID_to_lineage=np.load(this_file,allow_pickle=True).item()
        
    this_file=dirNameDataNapariLabels+"%s_%s_cell_lineages_dict_corrected.npy"%(trap,seg_chan)
            
    trap_roi_masks.append(roi_mask)
    trap_cellID_to_lineage.append(cellID_to_lineage)
print("Done!")



Loading Napari files...
Done!


## Select cells to visualize

In [17]:


for this_trap in trap_list:

    
    ##Filtered cells:
    #trackIDs_file=('%sdata/tracks_complete_info.csv'%rootDir)
    #df = pd.read_csv(trackIDs_file)
    #df_trackIDs_filtered=df.loc[(df['experiment_id'] == current_experiment) & (df['trap_id'] == this_trap)]
    #trackIDs=df_trackIDs_filtered['track_id'].values.tolist()

  
    #All cells
    trackIDs=df_lineages_geoms.trackID.unique()  
    #trackIDs=trackIDs[50:75] #dev
  

    print(this_trap,": Selected trackIDs ",len(trackIDs),':',trackIDs)


xy02 : Selected trackIDs  667 : [' 20.038-44.000' ' 20.090-44.001' ' 20.044-44.002' ' 38.004-44.003'
 ' 20.195-44.004' ' 20.073-44.005' ' 20.119-44.006' ' 20.068-44.007'
 ' 40.005-44.008' ' 20.037-44.009' ' 40.009-44.010' ' 42.007-44.011'
 ' 20.159-44.012' ' 26.049-44.013' ' 40.011-44.014' ' 20.112-44.015'
 ' 20.108-44.016' ' 41.006-44.017' ' 41.011-44.018' ' 43.007-44.019'
 ' 43.025-44.020' ' 20.025-44.021' ' 29.057-44.022' ' 32.049-44.023'
 ' 20.023-44.024' ' 39.019-44.025' ' 20.058-44.026' ' 20.043-44.027'
 ' 37.022-44.028' ' 34.055-44.029' ' 22.109-44.030' ' 36.013-44.031'
 ' 42.022-44.032' ' 20.052-44.033' ' 41.042-44.034' ' 41.033-44.035'
 ' 20.047-44.036' ' 42.040-44.037' ' 41.029-44.038' ' 39.025-44.039'
 ' 20.095-44.040' ' 20.179-44.041' ' 20.200-44.042' ' 20.077-44.043'
 ' 23.146-44.044' ' 43.035-44.045' ' 26.054-44.046' ' 20.085-44.047'
 ' 40.042-44.048' ' 27.106-44.049' ' 40.042-44.050' ' 41.044-44.051'
 ' 20.052-44.052' ' 20.173-44.053' ' 41.052-44.054' ' 41.057-44.055'
 '

In [18]:

def get_many_data_mask(tids, lbl_data):
    data_mask=deepcopy(empty_mask)
    for tid in tids:
        track_df=df_lineages_geoms[df_lineages_geoms["trackID"]==tid]
        for row in track_df.itertuples():
            this_poly=row.roiPoly
            if lbl_data is 'GFP':
                cell_data=float(row.GFP)
            elif lbl_data is 'DsRed':
                cell_data=float(row.DsRed)
            elif lbl_data is 'length':
                cell_data=float(row.length)
            elif lbl_data is 'division':
                cell_data=float(row.division)
            else:
                cname=tid.split('-')[0]
                cell_data=int(float(cname)*1e3) #TrackID
            
            cell_id=float(row.cellID)
            napari_coords=np.array([[int(y),int(x)] for (x,y) in this_poly.exterior.coords])
            mask = measure.grid_points_in_poly(matrix_shape, napari_coords)
            frame=row.frame-frame_experiment_start
            cname=tid.split('-')[0]
            data_mask[frame][mask] = cell_data
    return data_mask

In [19]:




def get_many_state_mask(tids, thres_DsRed, thres_length):
    state_mask=deepcopy(empty_mask)
    
    dead_tids=[]
    
    for tid in tids:
        track_df=df_lineages_geoms[df_lineages_geoms["trackID"]==tid]
        for row in track_df.itertuples():
            this_poly=row.roiPoly
            cell_id=float(row.cellID)
            cell_length=float(row.length)
            cell_DsRed=float(row.DsRed)
            cell_GFP=float(row.GFP)
            cell_state=float(row.state)
            
            #if tid in dead_tids:
            #    cell_state=1 #Dead
            #else:
            #    cell_state=2 #Alive
            #    if cell_DsRed>thres_DsRed:
            #        cell_state=1 #Dead
            #        if tid not in dead_tids:
            #            dead_tids.append(tid)
            #    else:
            #        if cell_length>thres_length:
            #            cell_state=3 #Elongated
            
            
            napari_coords=np.array([[int(y),int(x)] for (x,y) in this_poly.exterior.coords])
            mask = measure.grid_points_in_poly(matrix_shape, napari_coords)
            frame=row.frame-frame_experiment_start
            cname=tid.split('-')[0]
            state_mask[frame][mask] = cell_state
    return state_mask



def add_labels_state(trackIDs, this_trap, thres_DsRed, thres_length, dx, dy):
    #print("Drawing ",trackIDs)
    
    #clear_my_layers()
    #trackids=df_lineages_geoms.trackID.unique()
    #track_mask=get_many_data_mask(trackIDs, 'state')
    track_mask=get_many_state_mask(trackIDs, thres_DsRed, thres_length)
    
    color_state = {0: 'black', 1: '#629CFF', 2:'#FCF995',3:'#F8766D'}
    #track_mask=get_many_track_mask(trackIDs)
    trackLayer=cellViewer.add_labels(data=track_mask, name=this_trap+"_state", blending="additive", opacity=.9, translate=[dy, dx])
    trackLayer.color_mode = 'direct'
    trackLayer.color = color_state

    #LoadKeyBindedFuctions()
    cellViewer.window.activate()
    cellViewer.window.qt_viewer.console.execute()
   

In [20]:

def get_many_track_mask(tids):
    track_mask=deepcopy(empty_mask)
    for tid in tids:
        track_df=df_lineages_geoms[df_lineages_geoms["trackID"]==tid]
        for row in track_df.itertuples():#(index=False):
            this_poly=row.roiPoly
            cell_id=float(row.cellID)
            napari_coords=np.array([[int(y),int(x)] for (x,y) in this_poly.exterior.coords])
            mask = measure.grid_points_in_poly(matrix_shape, napari_coords)
            frame=row.frame-frame_experiment_start
            cname=tid.split('-')[0]
            int_track_mask=int(float(cname)*1e3)
            track_mask[frame][mask] = int_track_mask
    return track_mask

def add_labels_trackID(trackIDs, this_trap, dx, dy):
    #print("Drawing ",trackIDs)
    
    #clear_my_layers()
    #trackids=df_lineages_geoms.trackID.unique()
    
    track_mask=get_many_track_mask(trackIDs)
    trackLayer=cellViewer.add_labels(data=track_mask, color=colorsCid, name=this_trap+"_trackID", blending="additive", opacity=.9, translate=[dy, dx])
    trackLayer.color_mode = 'direct'
    trackLayer.color = colorsCid

    #LoadKeyBindedFuctions()
    cellViewer.window.activate()
    cellViewer.window.qt_viewer.console.execute()
    
    

In [21]:
def add_labels_data(trackIDs, this_trap, lbl_data, lbl_cmap, dx, dy, contrast_limits=[]):
    
    #print("Drawing",lbl_data,": ",trackIDs)
    
    data_mask=get_many_data_mask(trackIDs, lbl_data)
    if len(contrast_limits)==0:
        contrast_limits=[np.min(data_mask), np.max(data_mask)]
    print('contrast_limit=',contrast_limits)
        
    lbl_data='data_%s_%s'%(this_trap, lbl_data)
    
    #print(data_mask)
    #color = {-1: 'cyan', 0: 'yellow', -1: 'red'}
    #dataLayer=cellViewer.add_labels(data_mask, color=color, name=lbl_data, blending="additive", opacity=1)
    
    
    dataLayer=cellViewer.add_image(data_mask, colormap=lbl_cmap, name=lbl_data, blending="additive", opacity=1, contrast_limits=contrast_limits, translate=[dy, dx])
        
    #LoadKeyBindedFuctions()
    cellViewer.window.activate()
    cellViewer.window.qt_viewer.console.execute()
    
    
    

## Supplementary Figure 2

In [22]:

%gui qt

cellViewer=napari.Viewer()
cellViewer.title="Supp Movie 2"

trap_index=0
    
dx=0
dy1=-325
dy2=325

print(trap_tif_path[trap_index])

tif_path=trap_tif_path[trap_index]

#Top
cellViewer.open(tif_path+"GFP/",blending='additive',colormap="green",name=this_trap+"_GFP",visible=True,translate=[dx, dy1])
add_labels_data(trackIDs, this_trap, 'length', 'magenta', dy1, dx)
#add_labels_data(trackIDs, this_trap, 'GFP', 'green', dy1, dx)

#Botto
#cellViewer.open(tif_path+"DsRed/",blending='additive',colormap="magenta",name=this_trap+"_DsRed",visible=True,translate=[dx, dy1])
#add_labels_state(trackIDs, this_trap, thres_DsRed, thres_length, dy2, dx)        
    
#cellViewer.dims.set_axis_label(0,"frame(-%s)"%frame_experiment_start)


../../data/pBGT-Pulse/data_raw/xy02/
contrast_limit= [0.0, 228.87612582706075]
